In [2]:
import pandas as pd
import gzip
import json

In [17]:
# Se crean las rutas para cada uno de los archivos para lograr su posterior subida a pandas.
ruta_games = "/content/steam_games.json.gz"
ruta_reviews = "/content/user_reviews.json.gz"
ruta_items = "/content/users_items.json.gz"

In [18]:
# Verficar que elementos sean json válidos
def es_json_valido(cadena):
    try:
        json.loads(cadena)
        return True
    except ValueError as e:
        return False

# Revisión del archivo de steam_games

In [19]:
# Se carga el archivo de steam_games a pandas.
games = pd.read_json(ruta_games, compression = "gzip", lines = True)
games # se explora el archivo en pandas.

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [20]:
games.info() # se obtiene información sobre nulos y cantidad de datos "válidos" (datos no-null).

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


# Revisión del archivo de reviews

In [21]:
# Abre el archivo JSON.GZIP para descomprimirlo y convertirlo en una lista.
with gzip.open(ruta_reviews,"rt") as archivo_original:
  lineas_items = archivo_original.readlines()

# Se crea una lista donde se guardarán las líneas corregidas.
data_review = []

# Se revisa línea por línea y se reemplazan los datos que impiden que el JSON sea válido
for linea in lineas_items:
  linea = linea.replace('"',"'")
  linea = linea.replace("True","true").replace("False","false")
  linea = linea.replace("\': \'",'": "').replace("\', \'",'", "').replace("\': [{\'", '": [{"').replace("\': true, \'", '": true, "').replace("\': false, \'",'": false, "')
  linea = linea.replace("\'}",'"}').replace("\\\'","'").replace("{\'", '{"').replace("': []",'": []')
  linea = linea.strip('\n').strip("'")

  data_review.append(linea)


In [22]:
# Validar cada elemento de la lista, y se desagregan los elementos que no fueron posibles corregir a pesar de la limpieza realizada.
no_validos = []
reviews_validos = []

for elemento in data_review:
    if es_json_valido(elemento) == False:
        no_validos.append(elemento)
    else: reviews_validos.append(elemento)

# Número de elementos no válidos
print(f"EL total de elementos JSON no válidos de la data fue {len(no_validos)} de {len(data_review)}")

# Ya que el número de datos no válidos es poco, se decide continuar únicamente con los datos cuya limpieza fue posible realizar.

EL total de elementos JSON no válidos de la data fue 33 de 25799


In [23]:
# Convertir cada elemento de la lista a un diccionario
diccionario_reviews = [json.loads(elemento) for elemento in reviews_validos]

# Combinar los diccionarios en uno solo
json_reviews_corregido = json.dumps(diccionario_reviews, indent=2)

In [24]:
# Se convierten a un dataframe de pandas para revisar que la transformación sea exitosa.
reviews = pd.read_json(json_reviews_corregido)
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [25]:
reviews.info() # se obtiene información sobre nulos y cantidad de datos "válidos" (datos no-null).

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25766 entries, 0 to 25765
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25766 non-null  object
 1   user_url  25766 non-null  object
 2   reviews   25766 non-null  object
dtypes: object(3)
memory usage: 604.0+ KB


In [35]:
# Se vuelve a convertir en Json para comprimir y usar los archivos en una siguiente iteración.

js_reviews = reviews.to_json()
with gzip.open("/content/reviews_corregido.json.gz","w") as f:
  f.write(js_reviews.encode('utf-8'))

In [37]:
# se prueba que el nuevo archivo pueda ser leído al cargarse.
reviews = pd.read_json("/content/reviews_corregido.json.gz",compression="gzip")
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


# Revisión Items

In [26]:
# Abre el archivo JSON.GZIP para descomprimirlo y convertirlo en una lista.
with gzip.open(ruta_items,"rt") as archivo_original:
  lineas_items = archivo_original.readlines()

# Se crea una lista donde se guardarán las líneas corregidas.
data_items = []

# Se revisa línea por línea y se reemplazan los datos que impiden que el JSON sea válido
for linea in lineas_items:
  linea = linea.replace('"',"'")
  linea = linea.replace("True","true").replace("False","false")
  linea = linea.replace("\':",'":').replace(", \'", ', "').replace("{\'", '{"').replace(": \'",': "').replace("\',",'",')
  linea = linea.replace("\'}",'"}').replace("\\\'","'")
  linea = linea.strip('\n').strip("'")

  data_items.append(linea)

In [27]:
# Validar cada elemento de la lista, y se desagregan los elementos que no fueron posibles corregir a pesar de la limpieza realizada.
no_validos = []
items_validos = []

for elemento in data_items:
    if es_json_valido(elemento) == False:
        no_validos.append(elemento)
    else: items_validos.append(elemento)

# Número de elementos no válidos
print(f"EL total de elementos JSON no válidos de la data fue {len(no_validos)} de {len(data_items)}")

# Ya que el número de datos no válidos es poco, se decide continuar únicamente con los datos cuya limpieza fue posible realizar.

EL total de elementos JSON no válidos de la data fue 7 de 88310


In [28]:
# Convertir cada elemento de la lista a un diccionario
diccionario_items = [json.loads(elemento) for elemento in items_validos]

# Combinar los diccionarios en uno solo
json_items_corregido = json.dumps(diccionario_items, indent=2)

In [29]:
# Se convierten a un dataframe de pandas para revisar que la transformación sea exitosa.
items = pd.read_json(json_items_corregido)
items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [30]:
items.info() # se obtiene información sobre nulos y cantidad de datos "válidos" (datos no-null).

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88303 entries, 0 to 88302
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88303 non-null  object
 1   items_count  88303 non-null  int64 
 2   steam_id     88303 non-null  int64 
 3   user_url     88303 non-null  object
 4   items        88303 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.4+ MB


In [38]:
# Se vuelve a convertir en Json para comprimir y usar los archivos en una siguiente iteración.

js_items = items.to_json()
with gzip.open("/content/items_corregido.json.gz","w") as f:
  f.write(js_items.encode('utf-8'))

In [4]:
# se prueba que el nuevo archivo pueda ser leído al cargarse.
items = pd.read_json("/content/items_corregido.json.gz",compression="gzip")
items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
